In [2]:
#download catalogue from INGV

# %% LIB
from obspy.clients.fdsn.client import Client
from obspy import UTCDateTime
from obspy.core.event import Catalog
from obspy.core.stream import Stream
from obspy.core.event import Event
from obspy.core.event import Origin
from obspy.core.event import Magnitude
from obspy import read_events
from obspy import read_inventory
from obspy.core.event import ResourceIdentifier
import cartopy.crs as ccrs

from pyrocko import util, model, io, trace, moment_tensor, gmtpy
from pyrocko import pz
from pyrocko import orthodrome as od
from pyrocko.io import quakeml
from pyrocko.io import stationxml as fdsn
from pyrocko.client import catalog
from pyrocko.automap import Map

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import csv

In [ ]:
# rectangular search
workdir= '../'
catdir =  os.path.join(workdir,'CAT')

client=Client('INGV')

stime=UTCDateTime('1980-01-01T00:00:00')        # CHANGE set start time
etime=UTCDateTime('2014-01-01T00:00:00')        # CHANGE set end time

minlat=40.75                                       # CHANGE
maxlat=40.90                                       # CHANGE
minlon=14.00                                       # CHANGE
maxlon=14.20                                     # CHANGE

minmag=1.

catalogue_filename= 'catalogue_flegrei_1980_2014'              # CHANGE

print('downloading events from INGV database')
cat_INGV=client.get_events(starttime=stime,endtime=etime,
                        minlatitude=minlat,maxlatitude=maxlat,minlongitude=minlon,
                        maxlongitude=maxlon,minmagnitude=minmag)


cat_name=os.path.join(catdir,catalogue_filename + '.xml')
cat_INGV.write(cat_name,format='QUAKEML')

print('number of events selected:',len(cat_INGV))
print(cat_INGV)

downloading events from INGV database
number of events selected: 4
4 Event(s) in Catalog:
2010-12-21T02:47:12.000000Z | +40.845,  +14.069 | 1.3  ML | manual
2010-03-22T00:55:51.160000Z | +40.807,  +14.021 | 2.1  Md | manual
2000-08-22T19:07:49.860000Z | +40.771,  +14.070 | 2.5  Md | manual
1991-01-10T11:21:48.850000Z | +40.781,  +14.066 | 2.5  Md | manual


In [ ]:
# circular search
workdir= '../'
catdir =  os.path.join(workdir,'CAT')

client=Client('INGV')

stime=UTCDateTime('2020-01-01T00:00:00')        # CHANGE set start time
etime=UTCDateTime('2025-03-01T00:00:00')        # CHANGE set end time

lat=40.8478                                      # CHANGE
lon=14.0918                                      # CHANGE
minrad=0.1      # degrees
maxrad=4.     # degrees

maxdepth=50.    # km

minmag=4.5

catalogue_filename= 'catalogue_regional_events'              # CHANGE

print('downloading events from INGV database')
cat_INGV=client.get_events(starttime=stime,endtime=etime,
                        latitude=lat,longitude=lon,
                        minradius=minrad,maxradius=maxrad,
                        maxdepth=maxdepth,minmagnitude=minmag)


cat_name=os.path.join(catdir,catalogue_filename + '.xml')
cat_INGV.write(cat_name,format='QUAKEML')

print('number of events selected:',len(cat_INGV))
print(cat_INGV)

downloading events from INGV database
number of events selected: 17
17 Event(s) in Catalog:
2024-08-01T19:43:19.830000Z | +39.467,  +16.778 | 5.0  Mw | manual
2022-11-09T06:08:28.990000Z | +43.913,  +13.345 | 5.2  ML | manual
...
2016-08-24T02:33:28.890000Z | +42.792,  +13.151 | 5.4  Mw | manual
2016-08-24T01:36:32.000000Z | +42.698,  +13.233 | 6.0  Mw | manual
To see all events call 'print(CatalogObject.__str__(print_all=True))'


In [9]:
# create/load pf catalogue
catname_pf = os.path.join(catdir, catalogue_filename + '.pf')

def catalogue_INGV_to_PF(cat,cat_name):
    events = []
    for ev in cat:
        name= str(ev.resource_id).split('=')[1]

        timetmp= str(ev.origins[0].time)
        timestr = timetmp[0:10] + ' ' + timetmp[11:23]                          # format: '2010-01-01 00:00:00.000'
        time = util.str_to_time(timestr)

        lat=    float( ev.origins[0].latitude )
        lon =   float( ev.origins[0].longitude )
        depth = float( ev.origins[0].depth )
        magnitude = float( ev.magnitudes[0].mag )
        events.append(model.Event(name=name, time=time,
                                lat=lat, lon=lon,
                                depth=depth, magnitude=magnitude))
    events.sort(key=lambda x: x.time, reverse=False)
    model.dump_events(events, cat_name)
    return events

print('creating catalogues INGV in .pf')
cat_pf_INGV=    catalogue_INGV_to_PF(cat_INGV,catname_pf)

creating catalogues INGV in .pf


In [2]:
# %% EXTRA: create .txt file of the catalogue
workdir= '../'
catdir =  os.path.join(workdir,'CAT')
catalogue_filename= 'catalogue_flegrei_mag_2_5'            #CHANGE
file=os.path.join(catdir,catalogue_filename+'.pf')

def create_txt_catalogue(catalogue):
    list_events=[]
    for ev in catalogue:
        list_events.append( util.time_to_str(ev.time)+'\t'+str(ev.lat)
                           +'\t'+str(ev.lon)+'\t' +str(ev.depth)+'\t'+str(ev.magnitude) )
    return list_events

# columns:  TIME    LAT     LON     DEPTH   MAGNITUDE
######################################################################################
######################################################################################
switch_create_txt_file_with_catalogue= True      ########################SWITCH
######################################################################################
######################################################################################

if switch_create_txt_file_with_catalogue:
    print('Creating a txt file of the selected catalogue')
    cat=model.load_events(file)                                                         
    l_ev=create_txt_catalogue(cat)

    new_file = os.path.join(catdir,catalogue_filename + '.txt')

    with open(new_file, 'w') as output:
        for row in l_ev:
            output.write(str(row) + '\n')


# %% CHECK DATA NAME'S CONSISTENCIES:
# DATA
# BLACKLIST
# MARKERS

Creating a txt file of the selected catalogue
